# Dynamische Arrays

Wir haben gesehen, wie wir die Methoden ```append``` und ```pop```, die ein Element am Ende eines Arrays anfügen, respektive entfernen, effizient implementieren können. Hier nochmals die Implementation:

In [1]:
class  Array:
    
    def __init__(self):
        self.data = [None] # list  simulates  block  of  memory
        self.idx = 0  # index on next free element
        
    def append(self, elem):
        if self.idx == len(self.data):    
            self.resize(len(self.data) * 2)
        self.data[self.idx] = elem
        self.idx  += 1        
    
    def pop(self):
        self.idx -= 1
        item = self.data[self.idx]; 
        if self.idx > 0  and self.idx == len(self.data) / 4:
            self.resize(int(len(self.data) / 2)) 

        return item; 

    def resize(self, numElements):
        newArray = [None] * numElements

        for i in  range(0, self.idx):            
            newArray[i] = self.data[i]
        self.data = newArray
    
    def __str__(self):
        return str(self.data)
    
    def length(self):
        return self.idx


Die folgenden Tests zeigen, dass die Implementation wie erwartet funktioniert:

In [2]:
a = Array()
for i in range(0, 10):
    a.append(i)
print("Array after append: " + str(a))
print("length: " +str(a.length()))

print("idx before pop " +str(a.idx))
lastElement = a.pop()
print("lastElement " + str(lastElement))
print("Array after pop: " +str(a))
print("idx after pop " +str(a.idx))
print("length after pop: " +str(a.length()))

Array after append: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
length: 10
idx before pop 10
lastElement 9
Array after pop: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
idx after pop 9
length after pop: 9


#### Experimente

Wir wollen nun testen, wie sich die Laufzeit der append Methode verhält. 

In [86]:
import timeit
import random

In [87]:
def createByAppend(n):
    a = Array()
    for i in range(0, n):
        a.append(i)
    return a;

In [89]:
for n in [10, 100, 10000, 100000]:
    t = timeit.timeit(lambda: createByAppend(n), number=1)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Durchschnittliche Zeit für eine append Operation bei 10 Elementen = 1.629999997021514e-06
Durchschnittliche Zeit für eine append Operation bei 100 Elementen = 4.3399999867688164e-07
Durchschnittliche Zeit für eine append Operation bei 10000 Elementen = 3.8104000000203084e-07
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 3.724529999999504e-07


Wie wir sehen, ist die Durchschnittliche Zeit pro ```append```-Operation konstant, unabhängig davon, wieviele Elemente wir einfügen. 

#### Miniübung 

* Vergleichen Sie die Laufzeit mit einer naiven Implementation, bei der das Array bei jedem Aufruf vergrössert wird.

### Ammortisierte Analyse: Array resizing

Im Folgenden schauen wir uns die ```append``` Operation noch auf theoretischer Ebene an. Der folgende Code dient zum illustrieren der Idee der amortisierten Analyse. Die Idee ist einfach - wir führen Buchhalten. Bei jeder ```billigen``` Operation erhalten wir einen Betrag, mit dem wir später die teure Operation bezahlen. Unsere Währung ist dabei ein *Token*. Für jeden Arrayzugriff in der resize Methode bezahlen wir mit einem Token. Wenn wir es schaffen, dass wir bei jeder ```append```-Operation immer dieselbe konstante Anzahl Tokens auf die Seite legen können, und damit genügend Tokens zur Verfügung haben um damit die Anzahl Array-Zugriffe in der ```resize``` Operation bezahlen können, haben wir eine amortisiert konstante Laufzeit.

Im folgenden Code ist diese Idee umgesetzt:

In [83]:
class  ArrayWithAccounting:

    PRICE_IN_TOKENS_PER_APPEND = 5
    
    def __init__(self):
        self.data = [None] 
        self.idx = 0
        
        self.tokensSaved = 0
        self.tokensPaid = 0
        
    def append(self, elem):
        
        if self.idx == len(self.data):            
            newCapacity = len(self.data) * 2
            self.resize(newCapacity)        
        
        # Wir sparen eine Anzahl Tokens für jede Append Operation
        self.tokensSaved += self.PRICE_IN_TOKENS_PER_APPEND
        self.data[self.idx] = elem
        
        self.idx  += 1        

        

    def resize(self, numElements ):
        # Ein neues Array anzulegen braucht 1 Array Zugriffe pro Element
        # Wir bezahlen also ein Token pro Element
        newArray = [None] * numElements
        self.tokensPaid += 1 * numElements 
        
        for i in  range(0, self.idx):            
            newArray[i] = self.data[i]
            
            # Für jedes kopierte Element brauchen wir 2 Array Zugriffe 
            # Wir bezahlen also 2 Token pro Element
            self.tokensPaid += 2 
        self.data = newArray
    
    def __str__(self):
        return str(self.data)


Wenn wir diesen Code Ausführen, dann sehen wir, dass wir nach jeder Append Operation immer eine positive Anzahl Tokens haben. 

In [85]:
a = ArrayWithAccounting()
for i in range(0, 32):
    a.append(i)
print("paid ", a.tokensPaid, "saved ", a.tokensSaved)

paid  124 saved  160


#### Satz: Dynamisches vergrössern, verkleinern eines Arrays

Was wir nun experimentell herausgefunden haben lässt sich formal beweisen. Für eine Beweisskizze, siehe Sedgewick und Wayne, Algorithms, Satz E, Seite 221).

> Satz: Bei einem Array mit variabler Grösse ist gemäss obigen Algorithmus die durchschnittliche Anzahl der Arrayzugriffe für jede beliebige Folge von $M$ ```append``` und ```pop``` Operationen, ausgehend von dem leeren Array, proportional zu $M$.
